Author: Ao Chen, 26-12-2021, Sion.

In [35]:
import numpy as np
import pandas as pd
import xlwings as xw

In [36]:
xlsx_output = 1
json_output = 1
# sql_output  = 1 #specific sql engine should be defined 
filename_xlsx = '22122021CSC_Materials_DB.xlsx' #to be changed for different files
filename_json = '22122021CSC_Materials_DB.json' #same as above
# filename_sql  = '22122021CSC_Materials_DB.sql'

In [37]:
materials_db = pd.ExcelFile(filename_xlsx)
country_df = materials_db.parse('Country-list')
materials_df = materials_db.parse('Materials')

In [38]:
Country = country_df['Country']
Material = materials_df['Material']
Index = pd.MultiIndex.from_product([Country, Material], names=['Country', 'Material'])

In [39]:
kgCO2eq_kWh = country_df['1 kWh Medium voltage (kg CO2-Eq)']
energy_material = materials_df['Energy']
CO2eq = materials_df['CO2eq']
kgCO2eq = np.empty(len(Index)) # Initialization
ISO2_CC = np.empty(len(Index), dtype=object)
ISO3_CC = np.empty(len(Index), dtype=object)
Unit    = np.empty(len(Index), dtype=object)

In [41]:
for i in range(len(country_df)):
    for j in range(len(materials_df)):
        kgCO2eq[i*len(materials_df)+j] = kgCO2eq_kWh[i]*energy_material[j]+CO2eq[j]
        ISO2_CC[i*len(materials_df)+j] = country_df['ISO2_CC'][i]
        ISO3_CC[i*len(materials_df)+j] = country_df['ISO3_CC'][i]
        Unit[i*len(materials_df)+j]    = materials_df['Unit'][j]


In [42]:
Info = {'ISO2_CC': ISO2_CC, 'ISO3_CC': ISO3_CC, 'kgCO2eq': kgCO2eq, 'Unit': Unit,}
materialsDB_df = pd.DataFrame(data=Info)
materialsDB_df = materialsDB_df.set_index(Index)

In [48]:
if xlsx_output == 1:
    with pd.ExcelWriter(filename_xlsx, mode='a', if_sheet_exists='replace') as writer:
        materialsDB_df.to_excel(writer, sheet_name='Materials-DB')

In [57]:
if json_output == 1:
    materialsDB_df.to_json(filename_json, orient="table")